In [ ]:
import os
from huggingface_hub import login
from google.colab import drive

# 設置環境變數（從 Colab Secrets 讀取 HF_TOKEN）
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
if not hf_token:
    raise ValueError("HF_TOKEN not found. Please set it in Colab Secrets.")

# 登錄 Hugging Face
login(token=hf_token, add_to_git_credential=True)
print("Logged into Hugging Face successfully")

# 掛載 Google Drive（可選，若 MP4 在 Drive 中）
drive.mount('/content/drive')

# 設置檔案路徑（替換為您的 MP4 路徑）
mp4_path = "/content/drive/MyDrive/【原子少年專訪】曹家齊.mp4"
audio_path = "fenix_audio.wav" #設定本地路徑
output_dir = "fenix_segments_segmented" # 修改輸出目錄名稱以區分
video_output_dir = "fenix_video_segments_segmented" # 修改輸出目錄名稱以區分
data_dir = "data/fenix_segmented" # 修改資料目錄名稱以區分
temp_audio_segments_dir = "temp_audio_segments"

Logged into Hugging Face successfully
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyannote.audio pydub librosa torch soundfile huggingface_hub
!apt-get update -qq
!apt-get install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
from pydub import AudioSegment

# 提取音頻
try:
    video = AudioSegment.from_file(mp4_path, format="mp4")
    video = video.set_frame_rate(16000).set_channels(1) # 16kHz，單聲道
    video.export(audio_path, format="wav")
    print(f"Audio extracted to {audio_path}")
except Exception as e:
    raise Exception(f"Failed to extract audio: {str(e)}")

Audio extracted to fenix_audio.wav


In [ ]:
import torch
from pyannote.audio import Pipeline

# 載入語者分離模型
pipeline = None
try:
    pipeline = Pipeline.from_pretrained(
        "pyannote/speaker-diarization-3.1",
        use_auth_token=hf_token
    )
    if pipeline is None:
        raise ValueError("Failed to load pipeline. Check HF_TOKEN and model access.")

    if torch.cuda.is_available():
        print("CUDA is available. pyannote.audio might use it.")
    else:
        print("CUDA is not available. pyannote.audio will use CPU.")

except Exception as e:
    raise Exception(f"Error loading pipeline: {str(e)}")

config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

CUDA is available. pyannote.audio might use it.


In [ ]:
import os
import shutil
import subprocess
import soundfile as sf

def segment_audio(audio_path, num_segments=10, output_dir="temp_audio_segments"):
    """將音訊檔案分割成指定數量的片段。"""
    os.makedirs(output_dir, exist_ok=True)
    audio = AudioSegment.from_wav(audio_path)
    total_duration = len(audio)  # 以毫秒為單位
    segment_length = total_duration // num_segments
    segments = []
    for i in range(num_segments):
        start_time = i * segment_length
        end_time = (i + 1) * segment_length if i < num_segments - 1 else total_duration
        segment = audio[start_time:end_time]
        segment_filename = os.path.join(output_dir, f"segment_{i+1}.wav")
        segment.export(segment_filename, format="wav")
        segments.append((start_time / 1000, end_time / 1000, segment_filename)) # 保存開始/結束時間 (秒) 和檔案路徑
    return segments, total_duration / 1000 # 返回片段資訊和總時長 (秒)

def process_audio_segment(segment_path, pipeline):
    """對單個音訊片段執行語者分離。"""
    diarization = pipeline(segment_path)
    return diarization

if __name__ == "__main__":
    if 'pipeline' not in locals() or pipeline is None:
        print("Speaker diarization pipeline not loaded. Skipping segmentation.")
    else:
        num_segments = 100
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(video_output_dir, exist_ok=True)
        os.makedirs(data_dir, exist_ok=True)
        os.makedirs(temp_audio_segments_dir, exist_ok=True)

        audio_segments_info, total_duration = segment_audio(audio_path, num_segments)
        all_segments_result = []

        for i, (start_offset, end_offset, segment_path) in enumerate(audio_segments_info):
            print(f"Processing segment {i+1}/{num_segments} (from {start_offset:.2f}s to {end_offset:.2f}s)...")
            diarization_result = process_audio_segment(segment_path, pipeline)

            for segment, track, speaker in diarization_result.itertracks(yield_label=True):
                adjusted_start = segment.start + start_offset
                adjusted_end = segment.end + start_offset
                segment_filename = os.path.join(output_dir, f"{speaker}_{adjusted_start:.0f}-{adjusted_end:.0f}.wav")
                segment_audio = AudioSegment.from_wav(segment_path)[int(segment.start * 1000):int(segment.end * 1000)]
                segment_audio.export(segment_filename, format="wav")

                segment_info = {
                    "speaker": speaker,
                    "start": adjusted_start,
                    "end": adjusted_end,
                    "audio_path": segment_filename
                }

                if total_duration is not None and adjusted_end <= total_duration:
                    video_segment_filename = os.path.join(video_output_dir, f"{speaker}_{adjusted_start:.0f}-{adjusted_end:.0f}.mp4")
                    command = f"ffmpeg -i '{mp4_path}' -ss {adjusted_start} -to {adjusted_end} -c copy '{video_segment_filename}'"
                    try:
                        subprocess.run(command, shell=True, check=True, capture_output=True)
                        segment_info["video_path"] = video_segment_filename
                        print(f"  Extracted video segment: {video_segment_filename}")
                    except subprocess.CalledProcessError as e:
                        print(f"  Error extracting video segment: {e}")
                        segment_info["video_path"] = None
                else:
                    segment_info["video_path"] = None
                    print(f"  Warning: Video segment skipped for {segment_filename} due to duration mismatch.")

                all_segments_result.append(segment_info)

            print(f"Finished segment {i+1}/{num_segments}.")

        # 清理臨時音訊片段
        shutil.rmtree(temp_audio_segments_dir)

        # 將最終的分段資訊保存到 JSON 檔案
        import json
        segments_json_path = os.path.join(data_dir, "segments.json")
        with open(segments_json_path, 'w') as f:
            json.dump(all_segments_result, f, indent=4, ensure_ascii=False)
        print(f"Final segments information saved to {segments_json_path}")
        print("All steps completed with segmented processing!")

Processing segment 1/100 (from 0.00s to 3.77s)...
Finished segment 1/100.
Processing segment 2/100 (from 3.77s to 7.55s)...


/usr/local/lib/python3.11/dist-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


  Extracted video segment: fenix_video_segments_segmented/SPEAKER_00_4-7.mp4
Finished segment 2/100.
Processing segment 3/100 (from 7.55s to 11.32s)...
  Extracted video segment: fenix_video_segments_segmented/SPEAKER_00_8-8.mp4
  Extracted video segment: fenix_video_segments_segmented/SPEAKER_00_8-10.mp4
Finished segment 3/100.
Processing segment 4/100 (from 11.32s to 15.10s)...
Finished segment 4/100.
Processing segment 5/100 (from 15.10s to 18.88s)...
  Extracted video segment: fenix_video_segments_segmented/SPEAKER_00_16-19.mp4
  Extracted video segment: fenix_video_segments_segmented/SPEAKER_01_17-18.mp4
Finished segment 5/100.
Processing segment 6/100 (from 18.88s to 22.65s)...
  Extracted video segment: fenix_video_segments_segmented/SPEAKER_00_19-19.mp4
  Error extracting video segment: Command 'ffmpeg -i '/content/drive/MyDrive/【原子少年專訪】曹家齊.mp4' -ss 19.17596875 -to 19.44596875 -c copy 'fenix_video_segments_segmented/SPEAKER_00_19-19.mp4'' returned non-zero exit status 1.
  Extr

In [ ]:
import os
import shutil

# --- 將結果複製到 Google Drive (請確保之前的儲存格已成功執行完畢) ---
cloud_drive_path = "/content/drive/MyDrive/2025AWS/FEniX訓練資料2/" # 您的雲端路徑
output_dir_local = "fenix_segments_segmented" # 之前的程式碼使用這個本地輸出目錄
video_output_dir_local = "fenix_video_segments_segmented" # 之前的程式碼使用這個本地影片輸出目錄
data_dir_local = "data/fenix_segmented" # 之前的程式碼使用這個本地資料目錄

output_dir_cloud = os.path.join(cloud_drive_path, "audio_segments") # 雲端音訊輸出目錄
video_output_dir_cloud = os.path.join(cloud_drive_path, "video_segments") # 雲端影片輸出目錄
data_dir_cloud = os.path.join(cloud_drive_path, "data") # 雲端資料輸出目錄

os.makedirs(output_dir_cloud, exist_ok=True)
os.makedirs(video_output_dir_cloud, exist_ok=True)
os.makedirs(data_dir_cloud, exist_ok=True)

print(f"\nCopying results to Google Drive: {cloud_drive_path}")
try:
    shutil.copytree(output_dir_local, output_dir_cloud, dirs_exist_ok=True)
    print(f"Audio segments copied to: {output_dir_cloud}")
except Exception as e:
    print(f"Error copying audio segments: {e}")

try:
    shutil.copytree(video_output_dir_local, video_output_dir_cloud, dirs_exist_ok=True)
    print(f"Video segments copied to: {video_output_dir_cloud}")
except Exception as e:
    print(f"Error copying video segments: {e}")

try:
    shutil.copy(os.path.join(data_dir_local, "segments.json"), os.path.join(data_dir_cloud, "segments.json"))
    print(f"Segments JSON copied to: {os.path.join(data_dir_cloud, 'segments.json')}")
except Exception as e:
    print(f"Error copying segments JSON: {e}")

print("Finished copying results to Google Drive!")


Copying results to Google Drive: /content/drive/MyDrive/2025AWS/FEniX訓練資料2/
Audio segments copied to: /content/drive/MyDrive/2025AWS/FEniX訓練資料2/audio_segments
Video segments copied to: /content/drive/MyDrive/2025AWS/FEniX訓練資料2/video_segments
Segments JSON copied to: /content/drive/MyDrive/2025AWS/FEniX訓練資料2/data/segments.json
Finished copying results to Google Drive!
